# Testing with cu-dask

In [1]:
#cudf load
from dask_cuda import LocalCUDACluster
cluster=LocalCUDACluster()

NOTE: enter proxy/8787/status to get the dask dashboard - issue with kernels / conda / juypter

In [2]:
from dask.distributed import Client
#import dask.dataframe as dd
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:43213 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 62.81 GiB


### Loading our previous test data

In [4]:
import dask_cudf as dd

In [5]:
## reload our parquet
avg_bureau = dd.read_parquet('data_eng/avg_bureau.parq')
sum_cc_balance = dd.read_parquet(path='data_eng/sum_cc_balance.parq')
sum_payments = dd.read_parquet(path='data_eng/sum_payments.parq')
sum_pc_balance = dd.read_parquet(path='data_eng/sum_pc_balance.parq')
sum_prev = dd.read_parquet(path='data_eng/sum_prev.parq')
train = dd.read_csv('data/application_train.csv')

In [25]:
sum['SK_ID_CURR'].isnull().sum().compute()

0

In [27]:
# Merging the dataframes
## Lets try and merge everything with dask this time
train_feat_1 = train.drop('TARGET', axis=1) \
    .merge(avg_bureau, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR') \
    .merge(sum_cc_balance, how='left', left_on='SK_ID_CURR', right_index=True) 

In [28]:
train_feat_1.SK_ID_CURR.isnull().sum().compute()

0

In [30]:
train_feat__2 = train_feat_1.merge(sum_payments, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR') #\
    #.merge(sum_pc_balance, how='left', left_on='SK_ID_CURR', right_index=True) \
    #.merge(sum_prev, how='left', left_on='SK_ID_CURR', right_index=True) \
    #.drop('SK_ID_CURR', axis=1)

In [31]:
train_feat__2.head(10)

MemoryError: std::bad_alloc: CUDA error at: /opt/conda/envs/rapids/include/rmm/mr/device/cuda_memory_resource.hpp:69: cudaErrorMemoryAllocation out of memory